In [1]:
import numpy as np
from tqdm import tqdm
from scipy.stats import multivariate_normal
from scipy.stats import bernoulli
from scipy.integrate import quad

In [2]:
pi_theta = 1/3240.0
varcov_lambda = np.array(([0.25,0,0,0],[0,0.25,0,0],
                          [0,0,1,0], [0,0,0,1]))

In [3]:
nb_prior = 100

In [4]:
#scale parameters
#epsilon_t

mille = np.linspace(start=100, stop=1000, endpoint=True, num=10)[::-1]
cent = np.linspace(start=10, stop=100, endpoint=False, num=90)[::-1]
dix = np.linspace(start=5, stop=10, endpoint=False, num=10)[::-1]
cinq = np.linspace(start=3, stop=5, endpoint=False, num=40)[::-1]
trois = np.linspace(start=0, stop=3, endpoint=False, num=300)
trois = np.delete(arr=trois, obj=0)
trois = trois[::-1]

scale_param = np.concatenate((mille, cent, dix, cinq, trois))

In [5]:
prior_alpha = np.random.uniform(1.1, 2., size=1000)
prior_beta = np.random.uniform(-1., 1, size=1000)
prior_gamma = np.random.uniform(0., 30., size=1000)
prior_delta = np.random.uniform(-30., 30., size=1000)
prior_gen = np.vstack((prior_alpha,prior_beta,prior_gamma, prior_delta))
prior_gen = np.transpose(prior_gen)

In [6]:
def generation(n, alpha=1.7, beta=0.9, gamma=10, delta=10):
    # Initialize samples array with zeros
    sample = np.zeros(n)
    
    # Constants that do not depend on the sample index and thus can be computed once
    if alpha != 1:
        S_alpha_beta = (1 + beta ** 2 * np.tan(np.pi * alpha / 2) ** 2) ** (1 / (2 * alpha))
        B_alpha_beta = (1 / alpha) * np.arctan(beta * np.tan(np.pi * alpha / 2))

    for i in range(n):
        U = np.random.uniform(-np.pi/2, np.pi/2)
        W = -np.log(1 - np.random.uniform(0,1))
        
        # Handle the case alpha = 1 separately
        if alpha != 1:
            part1 = np.sin(alpha * (U + B_alpha_beta)) / (np.cos(U) ** (1 / alpha))
            part2 = (np.cos(U - alpha * (U + B_alpha_beta)) / W) ** ((1 - alpha) / alpha)
            sample[i] = S_alpha_beta * part1 * part2
        else:
            sample[i] = (2 / np.pi) * ((np.pi / 2 + beta * U) * np.tan(U) - beta * np.log((np.pi / 2 * W * np.cos(U))/(np.pi+beta*U)))

    # Apply scaling and location shifting
    sample = gamma * sample + delta
    return sample

In [7]:
def gaussian_ker(u=0, y=0, epsilon=1):
    """gaussian kernel for weights
    
    Parameters
    -------------------
    y : float, or array-like
    the point we have, the output

    u : float, or array-like
    the point from which we want to calculate a weight

    epsilon : int, float
    the scale parameter for which we want to compute the kernel
    ----------
    """

    w = (1/np.sqrt(2*np.pi*(epsilon**2)))*np.exp(-(np.abs((u-y)))**2/(2*(epsilon**2)))/epsilon
    return w

In [8]:
def zolotarev_transfo(sample, xi=0.25):
    """function to use for the estimation based on the zolotarev transformation

    Parameters
    --------------------------
    Sample : array-like
    Sample to do the transformation on

    xi : int, float
    The constant used in the transformation
    --------------------------
    """
    if xi<=0 or xi>1/2 :
        raise ValueError('Xi must be between 0 and 1/2')
    taille = len(sample)
    Z = []
    for i in range(1,int(taille/3)+1):
        transfo = sample[3*i-3] - xi*sample[3*i-2] - (1 - xi)*sample[3*i-1]
        Z.append(transfo)
    V = []
    U = []
    for i in range(len(Z)):
        V.append(np.log(np.abs(Z[i])))
        U.append(np.sign(sample[i]))
    V = np.array(V)
    U = np.array(U)
    S_U_squared = (np.std(U))**2
    S_V_squared = (np.std(V))**2
    nu_tilde = (6/(np.pi)**2)*S_V_squared - (3/2)*S_U_squared + 1
    etha_hat = np.mean(U)
    tau_hat = np.mean(V)
    nu_hat = 0
    if nu_tilde > ((1+np.abs(etha_hat))**2)/4:
        nu_hat = nu_tilde
    else:
        nu_hat = ((1+np.abs(etha_hat))**2)/4
    delta_hat = np.mean(sample)
    S_2 = np.array((nu_hat, etha_hat, tau_hat, delta_hat))
    return S_2

In [9]:
default = generation(n=1000)

In [10]:
def custom_integrand(u, epsilon_t):
    return np.exp(-u**2 / (2 * epsilon_t**2)) / (np.sqrt(2 * np.pi) * epsilon_t)

In [11]:
def pi_lf(epsilon_t, sample=default, alpha=1.7, beta=0.9, gamma=10, delta=10, possession=False):
    """function to have the likelihood free density given in the article, the data we create if there is
    no data provided is based on the parameters of 'generation', so for any comparaison between the values of theta,
    those are the parameters to change

    Parameters
    -----------------
    N : int
    number of priors we want to generate
        
    alpha, beta, gamma, delta : int, float
    the parameters with which we want to compute the true data if we don't already have it


    sample : array-like
    the data we observe

    summary_statistic : array-like
    summary statistic used to make tests on the distance between two datasets

    epsilon_t : int, float
    scale parameter, determines 

    Possession : boolean
    by default, set to false, to determine if we want to generate the data or if we already have it

    method : str
    the method with which we would compute the summary statistics
    -------------------
    """
    if possession == False :
        data = generation(n=1000)
    else :
        data = sample
    true_param = zolotarev_transfo(sample=data)
    proposal = generation(n=1000, alpha=alpha, beta=beta, gamma=gamma, delta=delta)
    zolo_proposal = zolotarev_transfo(sample=proposal)
    weight = gaussian_ker(u=np.linalg.norm(true_param-zolo_proposal), epsilon=epsilon_t)
    pi_lf = weight*pi_theta


    normalization_constant, _ = quad(custom_integrand, 0, np.inf, args=(epsilon_t,))


    pi_lf = pi_lf / normalization_constant

    return pi_lf
    
    return pi_lf

weight_gen[i] = pi_lf(epsilon_t=periode_t, theta_i, sample=y)/mutation_density(theta_i, t)

In [12]:
def mutation_creation(theta_t_1):
    """a function to sample a theta_t^(i) as in the paper, according to the distribution of the mutation
    kernel

    Parameters
    -------------------------
    theta_t_1 : array,
    the parameter at step t - 1
    --------------------------
    """
    weighted_random = np.zeros((nb_prior,4))
    for i in range(nb_prior):
        weighted_random[i] = weight_gen[i]*multivariate_normal.rvs(mean=theta_t_1, cov=varcov_lambda, size=1)
    mt_theta = np.sum(a=weighted_random, axis=0)
    return mt_theta

In [13]:
def mutation_density(theta_t, theta_t_1):
    """function that evaluates the pdf of the kernel for a given theta

    Parameters
    ----------------------
    theta_t : 1-D array, 
    the parameter of which we will estimate the probability density at step t

    theta_t_1 : array, 
    the parameter at a step before
    ----------------------
    """
    weighted_estimated = np.zeros(nb_prior)
    for i in range(nb_prior):
        weighted_estimated[i] = weight_gen[i]*multivariate_normal.pdf(theta_t, mean=theta_t_1,
                                                                      cov=varcov_lambda)
    mt_theta = np.sum(a=weighted_estimated, axis=0)
    return mt_theta
    

In [14]:
weight_gen = np.zeros((1000))

In [15]:
def resample(prior_gen=prior_gen, weight_gen=weight_gen, N=1000):
    """Resample N particles (rows) from 'prior_gen' matrix based on weights in 'weight_gen'.
    
    Parameters
    ------------------------
    prior_gen : 1-D or 2-D array,
    matrix of prior particles where each row is a particle

    weight_gen : 1-D or 2-D array 
    vector of weights corresponding to particles
    
    N : int,
    number of particles to resample.
    -------------------------
    """

    weights = weight_gen / np.sum(weight_gen)
    
    indices = np.random.choice(prior_gen.shape[0], size=N, replace=True, p=weights)
    
    resampled_particles = prior_gen[indices, :]
    
    return resampled_particles

### Initialization

In [34]:
nb_prior=3

In [35]:
y = generation(n=1000)

In [36]:
weight_gen = np.zeros((nb_prior))

In [37]:
prior_alpha = np.random.uniform(1.6, 1.7, size=nb_prior)
prior_beta = np.random.uniform(0.5, 1, size=nb_prior)
prior_gamma = np.random.uniform(9, 11, size=nb_prior)
prior_delta = np.random.uniform(9, 11, size=nb_prior)
prior_gen = np.vstack((prior_alpha,prior_beta,prior_gamma, prior_delta))
prior_gen = np.transpose(prior_gen)

In [38]:
for i in tqdm(range(nb_prior)):
    weight_gen[i] = pi_lf(epsilon_t=scale_param[0], alpha=prior_alpha[i],
                          beta=prior_beta[i], gamma=prior_gamma[i],
                          delta=prior_delta[i], possession=True, sample=y)/pi_theta
    

100%|██████████| 3/3 [00:00<00:00, 83.62it/s]


In [39]:
prior_gen=resample(prior_gen=prior_gen, weight_gen=weight_gen, N=nb_prior)

In [40]:
prior_t = prior_gen
prior_t_1 = prior_gen

In [41]:
weight_gen = np.ones(nb_prior)*1/nb_prior

In [42]:
for k in tqdm(range(10)):
    c_t = np.quantile(a=weight_gen, q=0.9) #we set the threshold as in the paper, before
    weight_gen = np.ones(nb_prior)*1/nb_prior
    prior_t_1 = prior_t
    for i in tqdm(range(nb_prior)):
        p_i = 0
        decision = 0
        compteur = 0
        while decision != 1 and compteur<100:
            theta_proposal = mutation_creation(prior_t_1[i])
            weight_gen[i] = pi_lf(possession=True, sample=y, epsilon_t=scale_param[k],
                                  alpha=theta_proposal[0], beta=theta_proposal[1],
                                  gamma=theta_proposal[2], delta=theta_proposal[2])/mutation_density(theta_t=prior_t[i],
                                                                                                     theta_t_1=prior_t_1[i])

            p_i = np.min((1,weight_gen[i]/c_t))

            if 0 < p_i < 1:
                decision = bernoulli.rvs(p_i, size=1)
                if decision == 1:
                    prior_t[i] = theta_proposal
                    weight_gen[i] = weight_gen[i]/p_i
            compteur += 1
            if compteur == 100:
                weight_gen[i] = 0

        prior_t = resample(prior_gen=prior_t, weight_gen=weight_gen, N=nb_prior)
        

  0%|          | 0/3 [00:00<?, ?it/s]/tmp/ipykernel_2722/3625153862.py:17: RuntimeWarning: invalid value encountered in scalar power
  part2 = (np.cos(U - alpha * (U + B_alpha_beta)) / W) ** ((1 - alpha) / alpha)

 33%|███▎      | 1/3 [00:01<00:02,  1.29s/it]/tmp/ipykernel_2722/3086114734.py:23: RuntimeWarning: divide by zero encountered in log
  V.append(np.log(np.abs(Z[i])))

 67%|██████▋   | 2/3 [00:02<00:01,  1.28s/it]/tmp/ipykernel_2722/3625153862.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  S_alpha_beta = (1 + beta ** 2 * np.tan(np.pi * alpha / 2) ** 2) ** (1 / (2 * alpha))
/tmp/ipykernel_2722/3625153862.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  B_alpha_beta = (1 / alpha) * np.arctan(beta * np.tan(np.pi * alpha / 2))
/tmp/ipykernel_2722/3625153862.py:8: RuntimeWarning: invalid value encountered in scalar multiply
  B_alpha_beta = (1 / alpha) * np.arctan(beta * np.tan(np.pi * alpha / 2))
/tmp/ipykernel_2722/3625153862.py:16: Runtime

ValueError: probabilities contain NaN

In [ ]:
scale_param[0]